## Ali's code

It seems that the way this link computes the weights of each topic in each document is not the optimal way (as it will not give you a weight if it is very small, but in big models, these small weights can also become important). Therefore instead, you can follow the following simple approach to get the weights:
To get the weights of a topic in the ith document, you call the method get_document_topics of the topic model object and pass the ith element of M1 (or tfidf_M1 in case you use tfidf) to it in the following way (minimum_probability=0 should be set so that none of near to zero entries are not removed) : 
lda_model.get_document_topics(M1[i],minimum_probability=0)
So let us see it in a complete example we had before: 

which gives the output (topics and weights of words in topics):

In [1]:
corpus = ["cryptography can be used for preventing data leakage in computer security",
"supervised learning and unsupervised learning are the two main groups of methods in machine learning",
"while in supervised learning we have access to the target variable in unsupervised learning we do not have such a variable",
"there are some methods in security for reducing the risk of information leakage like authentication and cryptography",
"topic modeling in an unsupervised machine learning model and therefore we do not have target variables"
]
stop_words = ["can","be","for","two","the","for","we","in","not","do","are","to","an","there","some","have","a","and","of","like","while","therefore","such"]

def clean_doc(doc):
    return " ".join([word for word in doc.lower().split() if word not in stop_words])

corpus_clean = [clean_doc(doc) for doc in corpus]
corpus_clean = [doc.split() for doc in corpus_clean]
from gensim import corpora
dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]
import gensim
Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(M1, num_topics=2, id2word = dictionary, passes=5,random_state =0)
topics = lda_model.print_topics(num_topics=5, num_words=10)
for topic in topics:
    print(topic)

(0, '0.084*"learning" + 0.071*"cryptography" + 0.071*"security" + 0.071*"leakage" + 0.070*"methods" + 0.043*"computer" + 0.043*"data" + 0.043*"used" + 0.042*"preventing" + 0.042*"authentication"')
(1, '0.136*"learning" + 0.093*"unsupervised" + 0.082*"target" + 0.080*"variable" + 0.058*"machine" + 0.057*"supervised" + 0.050*"topic" + 0.050*"variables" + 0.050*"modeling" + 0.050*"model"')


Now, the following call gives us the weight of the two topics in the first document:

In [2]:
lda_model.get_document_topics(M1[0],minimum_probability=0)

[(0, 0.9348568), (1, 0.06514323)]

And here is a complete working code with tfidf :
Note that as we expect, the weight should be different here as the model has been changed. 


In [6]:
corpus = ["cryptography can be used for preventing data leakage in computer security",
"supervised learning and unsupervised learning are the two main groups of methods in machine learning",
"while in supervised learning we have access to the target variable in unsupervised learning we do not have such a variable",
"there are some methods in security for reducing the risk of information leakage like authentication and cryptography",
"topic modeling in an unsupervised machine learning model and therefore we do not have target variables"
]
stop_words = ["can","be","for","two","the","for","we","in","not","do","are","to","an","there","some","have","a","and","of","like","while","therefore","such"]

def clean_doc(doc):
    return " ".join([word for word in doc.lower().split() if word not in stop_words])

corpus_clean = [clean_doc(doc) for doc in corpus]
corpus_clean = [doc.split() for doc in corpus_clean]
from gensim import corpora
dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]
import gensim
tfidf_model = gensim.models.TfidfModel(M1)
tfidf_M1 = tfidf_model[M1]
Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(tfidf_M1, num_topics=2, id2word = dictionary, passes=5,random_state =0)
topics = lda_model.print_topics(num_topics=5, num_words=10)
for topic in topics:
    print(topic)
lda_model.get_document_topics(tfidf_M1[0],minimum_probability=0)

(0, '0.058*"variable" + 0.055*"learning" + 0.046*"methods" + 0.045*"main" + 0.045*"cryptography" + 0.045*"groups" + 0.045*"security" + 0.045*"leakage" + 0.045*"supervised" + 0.043*"computer"')
(1, '0.060*"topic" + 0.059*"variables" + 0.059*"modeling" + 0.059*"model" + 0.050*"target" + 0.049*"machine" + 0.043*"learning" + 0.042*"unsupervised" + 0.035*"variable" + 0.035*"supervised"')


[(0, 0.83430034), (1, 0.16569969)]

# Our code

In [200]:
import nltk
import gensim
import re
from gensim import corpora
from gensim.parsing.preprocessing import *
#nltk.download('stopwords')      # If this code block complains, comment out this line
from nltk.corpus import stopwords
import pyLDAvis.gensim

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [201]:
def readFile(filename):
    fileData = ""
    file = open("abstracts/" + filename, 'r', encoding='utf8')
    fileData = file.read().splitlines()
    file.close
    return fileData

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [218]:
corpus = []
stop_words = stopwords.words('english')

CUSTOM_FILTERS = [
    strip_tags,                            # Remove HTML tags and such
    strip_punctuation,                     # Remove punctuation
    strip_multiple_whitespaces,            # Remove unecessary space and such
    strip_numeric,                         # Remove numbers
    lambda x: strip_short(x, 2),           # Remove words shorter than 2 chars
    lambda y: y.replace('“', ''),          # Removes quation marks
    lambda z: z.replace('”', ''),          # Removes quation marks
    lambda a: a.lower(),                   # Makes the corpus lowercase
    lambda b: remove_stopwords(b)          # Removes stopwords 
]

for i in range (2012,2020):
    conference = readFile('usenix' + str(i) + 'Abstracts.txt')
    for abstract in conference:
        corpus.append(abstract)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Preprocessing of the corpus

In [219]:
def clean_doc(doc):
    cleanDoc = preprocess_string(doc, CUSTOM_FILTERS)
    #cleanDoc = " ".join([word.lower() for word in cleanDoc if word not in stop_words])
    
    
    
    #cleanDoc = doc.replace('black', '')
    
    return cleanDoc

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cleaning the raw corpus documents

In [220]:
corpus_clean = [clean_doc(doc) for doc in corpus]


c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [221]:
corpus_clean

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['online',
  'sales',
  'counterfeit',
  'unauthorized',
  'products',
  'drive',
  'robust',
  'underground',
  'advertising',
  'industry',
  'includes',
  'email',
  'spam',
  'black',
  'hat',
  'search',
  'engine',
  'optimization',
  'forum',
  'abuse',
  'virtually',
  'encountered',
  'enticements',
  'purchase',
  'drugs',
  'prescription',
  'free',
  'online',
  'canadian',
  'pharmacy',
  'sites',
  'clearly',
  'economically',
  'motivated',
  'shape',
  'underlying',
  'business',
  'enterprise',
  'understood',
  'precisely',
  'underground',
  'paper',
  'exploit',
  'rare',
  'opportunity',
  'view',
  'organizations—the',
  'glavmed',
  'spamit',
  'rx',
  'promotion',
  'pharmaceutical',
  'affiliate',
  'programs—from',
  'inside',
  'ground',
  'truth',
  'data',
  'sets',
  'including',
  'years',
  'raw',
  'transaction',
  'logs',
  'covering',
  'million',
  'sales',
  'provide',
  'depth',
  'empirical',
  'analysis',
  'worldwide',
  'consumer',
  'demand',

Splitting the corpus and making it into a dictionary

In [223]:
#corpus_clean = [doc.split() for doc in corpus_clean]
dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Making the tfidf model

In [224]:
tfidf_model = gensim.models.TfidfModel(M1)
tfidf_M1 = tfidf_model[M1]

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Setting upp the lda model

In [229]:

Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(tfidf_M1, num_topics=5, id2word = dictionary, passes=5, random_state =0)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### All topics

In [230]:
topics = lda_model.print_topics(num_topics=5, num_words=10)

for topic in topics:
    print(topic)


(0, '0.001*"data" + 0.001*"security" + 0.001*"attacks" + 0.001*"code" + 0.001*"privacy" + 0.001*"memory" + 0.001*"kernel" + 0.001*"user" + 0.001*"access" + 0.001*"web"')
(1, '0.001*"attacks" + 0.001*"software" + 0.001*"malware" + 0.001*"security" + 0.001*"vulnerabilities" + 0.001*"attack" + 0.001*"cache" + 0.001*"analysis" + 0.001*"code" + 0.001*"control"')
(2, '0.001*"privacy" + 0.001*"web" + 0.001*"malware" + 0.001*"devices" + 0.001*"security" + 0.001*"attack" + 0.001*"data" + 0.001*"attacks" + 0.001*"users" + 0.001*"network"')
(3, '0.001*"attack" + 0.001*"attacks" + 0.001*"privacy" + 0.001*"users" + 0.001*"security" + 0.001*"network" + 0.001*"data" + 0.001*"censorship" + 0.001*"user" + 0.001*"password"')
(4, '0.001*"attacks" + 0.001*"memory" + 0.001*"security" + 0.001*"malware" + 0.001*"apps" + 0.001*"data" + 0.001*"password" + 0.001*"user" + 0.001*"analysis" + 0.001*"network"')


c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0, '0.001*"attacks" + 0.000*"attack" + 0.000*"security" + 0.000*"data" + 0.000*"cache" + 0.000*"memory" + 0.000*"privacy" + 0.000*"system" + 0.000*"user" + 0.000*"network"')
(1, '0.001*"security" + 0.001*"attacks" + 0.001*"memory" + 0.000*"apps" + 0.000*"web" + 0.000*"network" + 0.000*"attack" + 0.000*"data" + 0.000*"analysis" + 0.000*"code"')
(2, '0.001*"malware" + 0.001*"security" + 0.001*"privacy" + 0.000*"password" + 0.000*"network" + 0.000*"data" + 0.000*"use" + 0.000*"devices" + 0.000*"students" + 0.000*"users"')
(3, '0.001*"data" + 0.001*"attacks" + 0.001*"security" + 0.000*"users" + 0.000*"web" + 0.000*"code" + 0.000*"user" + 0.000*"vulnerabilities" + 0.000*"system" + 0.000*"attack"')

### Visualisation of topics

In [232]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, tfidf_M1, dictionary)
vis

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.014042 -0.005553       1        1  21.368423
4      0.007197  0.016221       2        1  21.295856
2     -0.004948 -0.012053       3        1  19.495958
0      0.000521 -0.004333       4        1  19.164269
3     -0.016812  0.005719       5        1  18.675494, topic_info=              Term      Freq     Total Category  logprob  loglift
3701           cfi  2.000000  2.000000  Default  30.0000  30.0000
2288      managers  1.000000  1.000000  Default  29.0000  29.0000
336       password  3.000000  3.000000  Default  28.0000  28.0000
610          cache  3.000000  3.000000  Default  27.0000  27.0000
2129  notification  1.000000  1.000000  Default  26.0000  26.0000
...            ...       ...       ...      ...      ...      ...
234          based  0.755808  4.613731   Topic5  -7.4623  -0.1310
154     techniques  0.729631  3.981021   Topic5  -7.4975  -0.0188
925           work  0.712258  3.549542   Topic5  -7.5216   0.0718
420         access  0.713554  3.919515   Topic5  -7.5198  -0.0255
5         analysis  0.714859  4.696594   Topic5  -7.5180  -0.2045

[398 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
420       1  0.255134  access
420       2  0.255134  access
420       3  0.255134  access
420       4  0.255134  access
420       5  0.255134  access
...     ...       ...     ...
925       1  0.281726    work
925       2  0.281726    work
925       3  0.281726    work
925       4  0.281726    work
925       5  0.281726    work

[355 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 1, 4])

### Topics per document

In [233]:
lda_model.get_document_topics(tfidf_M1[0],minimum_probability=0)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 0.02198147),
 (1, 0.021915898),
 (2, 0.022334285),
 (3, 0.021857508),
 (4, 0.9119108)]

In [141]:
s = "and I said \"Hello Bill\" "

c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [170]:
print(re.sub('"', '', s))
print(s)

and I said Hello Bill 
and I said "Hello Bill" 


c:\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
